# CS3033/CS6405 - Data Mining - Second Assignment

### Submission

This assignment is **due on 07/04/22 at 23:59**. You should submit a single .ipnyb file with your python code and analysis electronically via Canvas.
Please note that this assignment will account for 25 Marks of your module grade.

### Declaration

By submitting this assignment. I agree to the following:

<font color="red">“I have read and understand the UCC academic policy on plagiarism, and agree to the requirements set out thereby in relation to plagiarism and referencing. I confirm that I have referenced and acknowledged properly all sources used in the preparation of this assignment.
I declare that this assignment is entirely my own work based on my personal study. I further declare that I have not engaged the services of another to either assist me in, or complete this assignment”</font>

### Objective

The Boolean satisfiability (SAT) problem consists in determining whether a Boolean formula F is satisfiable or not. F is represented by a pair (X, C), where X is a set of Boolean variables and C is a set of clauses in Conjunctive Normal Form (CNF). Each clause is a disjunction of literals (a variable or its negation). This problem is one of the most widely studied combinatorial problems in computer science. It is the classic NP-complete problem. Over the past number of decades, a significant amount of research work has focused on solving SAT problems with both complete and incomplete solvers.

One of the most successful approaches is an algorithm portfolio, where a solver is selected among a set of candidates depending on the problem type. Your task is to create a classifier that takes as input the SAT instance's features and identifies the class.

In this project, we represent SAT problems with a vector of 327 features with general information about the problem, e.g., number of variables, number of clauses, the fraction of horn clauses in the problem, etc. There is no need to understand the features to be able to complete the assignment.


The original dataset is available at:
https://github.com/bprovanbessell/SATfeatPy/blob/main/features_csv/all_features.csv



## Data Preparation

In [24]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/andvise/DataAnalyticsDatasets/main/train_dataset.csv", index_col=0)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
df.shape

(2412, 310)

In [25]:
# Label or target variable
X = df.iloc[:,:-1]
y = df['target']


# Tasks

## Basic models and evaluation (5 Marks)

Using Scikit-learn, train and evaluate a decision tree classifier using 70% of the dataset from training and 30% for testing. For this part of the project, we are not interested in optimising the parameters; we just want to get an idea of the dataset.

In [26]:
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import tree

encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

train_features, test_features, train_labels, test_labels = model_selection.train_test_split(X,y, test_size=0.3, random_state=999)

cls = tree.DecisionTreeClassifier(random_state = 42)

cls.fit(train_features, train_labels)

cls.score(test_features, test_labels)

0.9765193370165746

## Robust evaluation (10 Marks)

In this section, we are interested in more rigorous techniques by implementing more sophisticated methods, for instance:
* Hold-out and cross-validation.
* Hyper-parameter tuning.
* Feature reduction.
* Feature selection.
* Feature normalisation.

Your report should provide concrete information about your reasoning; everything should be well-explained.

The key to geting good marks is to show that you evaluated different methods and that you correctly selected the configuration.

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Data is split into train and test features and labels. 70% Train size set, 30% Test size set in random state 999
train_features, test_features, train_labels, test_labels = model_selection.train_test_split(X,y, test_size=0.3, random_state=999)


#MinMax scaler, Principle Component Analysis and Decision Tree Classifier are defined and combined into pipeline.
scaler = MinMaxScaler()
pca = PCA()
cls = tree.DecisionTreeClassifier(random_state = 42)
pipe = Pipeline([('scaler', MinMaxScaler()), ('pca', PCA()), ('cls', tree.DecisionTreeClassifier())])


#Parameters for grid searchCV, list needed to be wide enough to catch best parameters but narrow enough to be practical to run. Parameters changed multiple times, some attempts shown
#in cell below
#OPTIMAL VALUES -- PCA_n_components: 15 ,  cls_criterion: Entropy,   cls__max_depth: None,     cls__min_samples_split: 2,       pca__whiten: False
param_grid = {'pca__n_components': [15],
              'pca__whiten': [False],
              'cls__criterion': ['entropy'],
              'cls__max_depth': [None],
              'cls__min_samples_split': [2],
              }


#Define the GridSearch by passing, the pipeline and the parameter grid, perform 5 fold stratified corss validation.(Data is split again using now using three sets)
#Fit/Train the grid on the training data 
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(train_features, train_labels)


#Print best classifier, hyperparaters and training score
print("The best classifier is:", grid.best_estimator_)
print("Best hyper-parameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)



y_pred = grid.best_estimator_.predict(test_features)

# Compute the accuracy score of the model
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy score: ", accuracy)

 

#https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html 
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

The best classifier is: Pipeline(steps=[('scaler', MinMaxScaler()), ('pca', PCA(n_components=15)),
                ('cls', DecisionTreeClassifier(criterion='entropy'))])
Best hyper-parameters:  {'cls__criterion': 'entropy', 'cls__max_depth': None, 'cls__min_samples_split': 2, 'pca__n_components': 15, 'pca__whiten': False}
Best score:  0.952002528400611
Accuracy score:  0.93646408839779


In [ ]:
param_grid = {'pca__n_components': [20, 25,15,60,45],
              'cls__criterion': ['entropy','gini'],
              'cls__max_depth': [None],
              'cls__min_samples_split': [2,4,10]}

param_grid = {'pca__n_components': [20, 25,15,60,45],
              'cls__criterion': ['entropy'],
              'cls__max_depth': [None],
              'cls__min_samples_split': [2]}

param_grid = {'pca__n_components': [2, 5, 10, 20, 30, 40],
              'cls__criterion': ['gini', 'entropy'],
              'cls__max_depth': [None, 5, 10],
              'cls__min_samples_split': [2, 5, 10]}

param_grid = {'pca__n_components': [15],
              'pca__whiten': [True, False],
              'cls__criterion': ['entropy'],
              'cls__max_depth': [None],
              'cls__min_samples_split': [2],
              }



## New classifier (10 Marks)

Replicate the previous task for a classifier different than K-NN and decision trees. Briefly describe your choice.
Try to create the best model for the given dataset.


Save your best model into your github. And create a single code cell that loads it and evaluate it on the following test dataset:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/test_dataset.csv

This link currently contains a sample of the training set. The real test set will be released after the submission. I should be able to run the code cell independently, load all the libraries you need as well.

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import StandardScaler

#Data Split as before

train_features, test_features, train_labels, test_labels = model_selection.train_test_split(X,y, test_size=0.3, random_state=999)

#Define a new Pipeline using the RandomForest Classifier 
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('select_k_best', SelectKBest()),
    ('rf', RandomForestClassifier())
])

#Made a wide and broad param grid to captrure as much as possible, I let it run for several hours to find the best combination.

#Best hyper-parameters:  {'pca__n_components': 30, 'pca__whiten': True, 'rf__bootstrap': False, 
                         #'rf__criterion': 'entropy', 'rf__max_depth': None, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 
                         ##'rf__n_estimators': 200, 'scaler': StandardScaler(), 'select_k_best__k': 20, 'select_k_best__score_func': <function f_classif at 0x7f99a47d8670>}
param_grid = {
    'scaler': [StandardScaler()],
    'pca__n_components': [30],
    'pca__whiten': [True],
    'select_k_best__k': [20],
    'select_k_best__score_func': [mutual_info_classif],
    'rf__n_estimators': [200],
    'rf__criterion': ['entropy'],
    'rf__max_features': ['sqrt'],
    'rf__max_depth': [None],
    'rf__min_samples_split': [2],
    'rf__min_samples_leaf': [2],
    'rf__bootstrap': [False]
}

# Define the GridSearchCV object - discovered i could use the n_jobs =-1 to allow all my processors cores to be used to try and speed up the process, 5 fold cross validation
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(train_features, train_labels)

#print the best combination
print("The best classifier is:", grid.best_estimator_)
print("Best hyper-parameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)

y_pred = grid.best_estimator_.predict(test_features)

# Compute the accuracy score of the model
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy score: ", accuracy)     


#https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier

The best classifier is: Pipeline(steps=[('scaler', StandardScaler()),
                ('pca', PCA(n_components=30, whiten=True)),
                ('select_k_best',
                 SelectKBest(k=20,
                             score_func=<function mutual_info_classif at 0x7f99a47ea280>)),
                ('rf',
                 RandomForestClassifier(bootstrap=False, criterion='entropy',
                                        min_samples_leaf=2,
                                        n_estimators=200))])
Best hyper-parameters:  {'pca__n_components': 30, 'pca__whiten': True, 'rf__bootstrap': False, 'rf__criterion': 'entropy', 'rf__max_depth': None, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 2, 'rf__n_estimators': 200, 'scaler': StandardScaler(), 'select_k_best__k': 20, 'select_k_best__score_func': <function mutual_info_classif at 0x7f99a47ea280>}
Best score:  0.9905220093761523
Accuracy score:  0.9834254143646409


In [ ]:
Original_param_grid = {
    'scaler': [StandardScaler(), MinMaxScaler()],
    'pca__n_components': [10, 24, 30],
    'pca__whiten': [True, False],
    'select_k_best__k': [5, 10, 20],
    'select_k_best__score_func': [f_classif, mutual_info_classif],
    'rf__n_estimators': [100, 200, 500],
    'rf__criterion': ['entropy'],
    'rf__max_features': ['auto', 'sqrt', 'log2'],
    'rf__max_depth': [None],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [2],
    'rf__bootstrap': [True, False]
}

# <font color="blue">FOR GRADING ONLY</font>

Save your best model into your github. And create a single code cell that loads it and evaluate it on the following test dataset: 
https://github.com/andvise/DataAnalyticsDatasets/blob/main/test_dataset.csv


In [8]:
from joblib import dump, load
from io import BytesIO
import requests
import pandas as pd
import numpy as np

mLink = 'https://github.com/JazzLiberal/DataMining/blob/main/model122108729.joblib?raw=true'
mfile = BytesIO(requests.get(mLink).content)
model = load(mfile)

# Load the dataset and replicate your preprocessing
df = pd.read_csv("https://github.com/andvise/DataAnalyticsDatasets/blob/main/test_dataset.csv?raw=true", index_col=0)
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)


X = df.iloc[:,:-1]
y = df['target']

encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

#Data Split as before

train_features, test_features, train_labels, test_labels = model_selection.train_test_split(X,y, test_size=0.3, random_state=122)

# Evaluate your model or pipeline
model.score(test_features, test_labels)  

KeyError: ignored